In [1]:
import sys

sys.path.insert(0, "..")
import csv
import os
import cocpit.config as config
import cocpit
from ai2es import gui_test_loader
import ipywidgets
import pandas as pd
import torch
from PIL import ImagePath
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

If checking classifications from realtime predictions in csv:

In [9]:
date = '2022/08/06'
pred_dir = '/ai2es/realtime_predictions/csv'
pred = 'precipitation'
file_list = []
open_dir = []
with open(f"{pred_dir}/{date}/{date.replace('/','_')}.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        file_list.append(row[0])
        station = row[0].split('.')[0].split('_')[1]
        open_dir.append(f'/ai2es/cam_photos/{date}/{station}')

if checking classifications from test folder:

In [ ]:
#station_id = 'VOOR'
#open_dir = f"/ai2es/test_set/{station_id}"
open_dir = '/ai2es/test_set/shap/'
file_list = os.listdir(open_dir)

In [10]:
test_data = cocpit.data_loaders.TestDataSet(open_dir, file_list)
test_loader = cocpit.data_loaders.create_loader(test_data, batch_size=100, sampler=None)
p = cocpit.predictions.LoaderPredictions()
with torch.no_grad():
    for fold in range(config.KFOLD+1):
        model = p.load_model(fold)
        for (imgs, paths) in test_loader:
            b = cocpit.predictions.BatchPredictions(imgs, model)
            b.find_max_preds()
            b.top_k_preds(len(config.CLASS_NAMES))
            p.append_batch(b, paths)
p.concatenate_loader_vars()

In [3]:
# for each file in the test loader,
# display 1 min accumulated precip quantity 
# siftfing through all of the images to find files in test set
df = pd.read_parquet("/ai2es/matched_parquet/2017.parquet")
search_paths = df['path'].str.split('/')
search_paths = search_paths.apply(lambda x: x[-1])
precip = [df['precip_accum_1min [mm]'][search_paths == file] for file in file_list]

In [11]:
gui = gui_test_loader.GUI(p.paths, p.topk_probs, p.topk_classes, precip=None)
gui.interp()
display(ipywidgets.HBox([gui.output, gui.next_btn]))

Make a prediction on one image

In [4]:
import torch
import cv2
from torchvision.transforms import Compose, Normalize, ToTensor, Resize
import numpy as np
from cocpit import config as config
import torch.nn.functional as F
import torchvision.transforms as T

def preprocess_image(
    img: np.ndarray, mean, std) -> torch.Tensor:
    preprocessing = Compose([
        ToTensor(),
        Resize((224,224)),
        Normalize(mean=mean, std=std)
    ])
    return preprocessing(img.copy()).unsqueeze(0)


model = torch.load("/ai2es/saved_models/v0.0.0/e[30]_bs[64]_k0_1model(s).pt")
model.eval()
rgb_img = cv2.imread('/ai2es/cam_photos/2017/04/02/CLAR/20170402T002001_CLAR.jpg', 1)[:, :, ::-1]
rgb_img = np.float32(rgb_img) / 255
input_tensor = preprocess_image(rgb_img,
                                mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])

outputs=model(input_tensor)
preds = F.softmax(outputs, dim=1)
_, max_preds = torch.max(outputs, 1)
max_preds = max_preds.cpu()
preds, max_preds

(tensor([[9.5876e-02, 1.3070e-05, 9.0411e-01]], device='cuda:0',
        grad_fn=<SoftmaxBackward>),
 tensor([2]))